In [1]:
from pyomo.environ import ConcreteModel, SolverFactory, value, Constraint, Var, Reference, units, Param 
from pyomo.core.base.var import VarData   # import once, at top

from idaes.core import FlowsheetBlock, MaterialBalanceType, MomentumBalanceType
from idaes.models.unit_models import Translator, Separator, Mixer
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock
from idaes.models_extra.power_generation.properties.natural_gas_PR import get_prop, get_rxn, EosType
import pyomo.environ as pyo
import idaes.core.util.scaling as iscale
import idaes.core.util.model_statistics as stattools
from idaes.core.util.initialization import propagate_state
import idaes.logger as idaeslog
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.models.unit_models as um
from pyomo.opt import TerminationCondition        #  ← add this import


from idaes.core.util.model_statistics import (
    degrees_of_freedom, 
    report_statistics,
)
from pyomo.network import Arc
from idaes.models.properties.modular_properties.base.generic_reaction import GenericReactionParameterBlock

In [2]:
# Logger silenzioso
idaeslog.getLogger("idaes.init").setLevel(idaeslog.ERROR)
idaeslog.getLogger("idaes.solve").setLevel(idaeslog.ERROR)

# Modello
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Proprietà
fuel_comps = {"H2": 0.99, "H2O": 0.01}
air_comps = {"O2": 0.21, "N2": 0.79}
reaction_comps = {"H2", "H2O", "O2"}

m.fs.fuel_props = GenericParameterBlock(**get_prop(fuel_comps, phases={"Vap"}, eos=EosType.PR))
m.fs.air_props = GenericParameterBlock(**get_prop(air_comps, phases={"Vap"}, eos=EosType.PR))
m.fs.reaction_props = GenericParameterBlock(**get_prop(reaction_comps, phases={"Vap"}, eos=EosType.PR))

def print_stream(unit, label, stream):
    separator = '=' * 60
    subsection = '-' * 60

    def format_var(i, var):
        try:
            val = value(var)
            val_str = f"{val: .6e}" if abs(val) < 1e5 else f"{val: .3f}"
        except:
            val_str = "[NaN]"
        return f"{i:>3} - {val_str:>14}   {var}"

    print(f"\n{separator}")
    print(f"{label:^60}")
    print(f"{separator}")

    print("\nSTREAM PROPERTIES:")
    print(f"{'Flow [mol/s]':<20}: {value(stream.flow_mol[0]):.6e}")
    print(f"{'Temperature [K]':<20}: {value(stream.temperature[0]):.2f}")
    print(f"{'Pressure [Pa]':<20}: {value(stream.pressure[0]):.2f}")

    print("\nMOLE FRACTIONS:")
    for t, c in stream.mole_frac_comp.keys():
        print(f"  x_{c:<10}: {value(stream.mole_frac_comp[t, c]):.6e}")

    print(f"\nDOF after {label}: {degrees_of_freedom(unit)}")

    # print(f"\n{subsection}")
    # print("MODEL STATISTICS REPORT:")
    # print(subsection)
    # report_statistics(unit)

    # print(f"\n{subsection}")
    # print("VARIABLES:")
    # print(subsection)
    # for i, v in enumerate(variables_set(unit)):
    #     print(format_var(i + 1, v))

    # print(f"\n{subsection}")
    # print("ACTIVATED CONSTRAINTS:")
    # print(subsection)
    # for i, c in enumerate(activated_constraints_set(unit)):
    #     print(f"{i + 1:>3} - {c}")

    # print(f"\n{subsection}")
    # print("ACTIVATED EQUALITIES:")
    # print(subsection)
    # for i, eq in enumerate(activated_equalities_set(unit)):
    #     print(f"{i + 1:>3} - {eq}")

    # print(f"\n{subsection}")
    # print("UNFIXED VARIABLES:")
    # print(subsection)
    # for i, ufv in enumerate(unfixed_variables_set(unit)):
    #     print(format_var(i + 1, ufv))

    # print(f"\n{subsection}")
    # print("FIXED VARIABLES:")
    # print(subsection)
    # for i, fv in enumerate(fixed_variables_set(unit)):
    #     print(format_var(i + 1, fv))

In [3]:
# ==============================================================
# 1) Translator H2
# ==============================================================
m.fs.translator_h2 = Translator(
    inlet_property_package=m.fs.fuel_props,
    outlet_property_package=m.fs.reaction_props,
    outlet_state_defined=False,
)
m.fs.translator_h2.inlet.flow_mol[0].fix(0.075397)
m.fs.translator_h2.inlet.temperature[0].fix(942.05)
m.fs.translator_h2.inlet.pressure[0].fix(101325)
m.fs.translator_h2.inlet.mole_frac_comp[:, "H2"].fix(0.99)
m.fs.translator_h2.inlet.mole_frac_comp[:, "H2O"].fix(0.01)

@m.fs.translator_h2.Constraint(m.fs.time)
def temp_eqn_h2(b, t): 
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_h2.Constraint(m.fs.time)
def press_eqn_h2(b, t): 
    return b.properties_out[t].pressure == b.properties_in[t].pressure

@m.fs.translator_h2.Constraint(m.fs.time)
def flow_eqn_h2(b, t): 
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_h2.Constraint(m.fs.time, list(m.fs.translator_h2.properties_in[0].mole_frac_comp.keys()))
def mole_frac_eqn_h2(b, t, j): 
    return b.properties_out[t].mole_frac_comp[j] == b.properties_in[t].mole_frac_comp[j]
# ==============================================================
# 2) Air Separator
# ==============================================================
m.fs.separator = Separator(
    property_package=m.fs.air_props,
    outlet_list=["o2_rich_strm", "o2_poor_strm"],
    split_basis=um.SplittingType.componentFlow,
    momentum_balance_type = MomentumBalanceType.pressureTotal,
    
)
m.fs.separator.inlet.flow_mol[0].fix(2)
m.fs.separator.inlet.temperature[0].fix(948.45)
m.fs.separator.inlet.pressure[0].fix(101325) 
m.fs.separator.inlet.mole_frac_comp[0, "O2"].fix(0.21)
m.fs.separator.inlet.mole_frac_comp[0, "N2"].fix(0.79)


m.fs.separator.split_fraction[:, "o2_rich_strm", "O2"].fix(1)
m.fs.separator.split_fraction[:, "o2_poor_strm", "N2"].fix(1)
# ==============================================================
# 3) Translator O2
# ==============================================================
m.fs.translator_o2 = Translator(
    inlet_property_package=m.fs.air_props,
    outlet_property_package=m.fs.reaction_props,
    outlet_state_defined=True,
)
m.fs.o2_to_translator = Arc(source=m.fs.separator.o2_rich_strm, destination=m.fs.translator_o2.inlet)

comps_out = set(m.fs.translator_o2.properties_out[0].mole_frac_comp.keys())

@m.fs.translator_o2.Constraint(m.fs.time, comps_out)
def mole_frac_eqn_o2_out(b, t, j):
    if j == "O2":
        return b.properties_out[t].mole_frac_comp[j] == b.properties_in[t].mole_frac_comp[j]
    else: 
        return b.properties_out[t].mole_frac_comp[j] == 1e-19
    
@m.fs.translator_o2.Constraint(m.fs.time)
def flow_eqn_o2(b, t): 
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol 

@m.fs.translator_o2.Constraint(m.fs.time)
def temp_eqn_o2(b, t): 
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_o2.Constraint(m.fs.time)
def press_eqn_o2(b, t): 
    return b.properties_out[t].pressure == b.properties_in[t].pressure

# Le variabili d’ingresso (flow_mol, T, P, x_O2, x_N2) non sono fissate dentro il Translator: 
# sono semplicemente ugualiate (via l’Arc) alle corrispondenti variabili della corrente o2_rich_strm 
# del Separator. Dal punto di vista interno al blocco, quindi, quei 5 simboli rimangono “unfixed” ⇒ 5 DoF.
# I cinque gradi di libertà del Translator saranno compensati dai vincoli e dai fix impostati negli altri
# blocchi (il Separator in particolare) dopo che il modello sarà risolto con solver.
# ==============================================================
# 4) Mixer (feed)
# ==============================================================
m.fs.mixer = um.Mixer(
    inlet_list=["fuel_strm","o2_rich_strm"],
    momentum_mixing_type=um.MomentumMixingType.none,
    property_package=m.fs.reaction_props,
)
m.fs.mix_h2 = Arc(source=m.fs.translator_h2.outlet, destination=m.fs.mixer.fuel_strm)
m.fs.mix_o2 = Arc(source=m.fs.translator_o2.outlet, destination=m.fs.mixer.o2_rich_strm)


@m.fs.mixer.Constraint(m.fs.time)
def pressure_eqn(b, t):
    return b.mixed_state[t].pressure == b.fuel_strm_state[t].pressure
# ==============================================================
# 5) Stoichiometric Reactor
# ==============================================================
m.fs.rxn_props = GenericReactionParameterBlock(**get_rxn(m.fs.reaction_props, {"h2_cmb"}))
m.fs.reactor = um.StoichiometricReactor(
    property_package=m.fs.reaction_props,
    reaction_package=m.fs.rxn_props,
    has_pressure_change=False,
    has_heat_transfer= True, #Luca: should be false
)

m.fs.mix_to_reactor = Arc(source=m.fs.mixer.outlet, destination=m.fs.reactor.inlet)

m.fs.fuel_util = Var(
        m.fs.time,
        initialize=0.7,
        bounds=(0, 1),          # optional but convenient
        units=units.dimensionless,
        doc="Fuel-utilisation factor",
)
m.fs.fuel_util.fix(0.7)   # 70 % conversion

# 2) use it inside the constraint
@m.fs.reactor.Constraint(m.fs.time)
def fuel_utilization_eqn(b, t):
    # note the explicit time index on flow_mol_comp
    return (
        b.control_volume.properties_out[t].flow_mol_comp["H2"]
        == b.control_volume.properties_in[t].flow_mol_comp["H2"]
           * (1 - m.fs.fuel_util[t])
    )
# ==============================================================
# 6) Product Separator
# ==============================================================
m.fs.reactor_separator = Separator(
    property_package=m.fs.reaction_props,
    outlet_list=['water_strm', 'o2_strm'],
    split_basis=um.SplittingType.componentFlow,
    material_balance_type= MaterialBalanceType.componentPhase,
    momentum_balance_type = MomentumBalanceType.pressureTotal,
)
m.fs.react_to_sep = Arc(source=m.fs.reactor.outlet, destination=m.fs.reactor_separator.inlet)


m.fs.reactor_separator.split_fraction[:, "o2_strm", "O2"].fix(1)
m.fs.reactor_separator.split_fraction[:, "o2_strm", "H2"].fix(0)
m.fs.reactor_separator.split_fraction[:, "o2_strm", "H2O"].fix(0)
# ==============================================================
# 7) Translator H2_out
# ==============================================================
m.fs.translator_h2_out = Translator(
    inlet_property_package=m.fs.reaction_props,
    outlet_property_package=m.fs.fuel_props,
    outlet_state_defined=False,
)

# Connessione dallo stream "water_strm" del separatore
m.fs.water_to_translator = Arc(
    source=m.fs.reactor_separator.water_strm,
    destination=m.fs.translator_h2_out.inlet
)


# Constraint FTP standard (totale)
@m.fs.translator_h2_out.Constraint(m.fs.time)
def flow_eqn_h2_out(b, t):
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_h2_out.Constraint(m.fs.time)
def temp_eqn_h2_out(b, t):
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_h2_out.Constraint(m.fs.time)
def press_eqn_h2_out(b, t):
    return b.properties_out[t].pressure == b.properties_in[t].pressure

# Componenti comuni
comps_out = set(m.fs.translator_h2_out.properties_out[0].mole_frac_comp.keys())
comps_out.remove("H2")
# SOLO constraint flussi molari individuali per componenti comuni
@m.fs.translator_h2_out.Constraint(m.fs.time, comps_out)
def component_flow_eqn_h2_out(b, t, j):
    return (
                b.properties_out[t].mole_frac_comp[j]
                == b.properties_in[t].mole_frac_comp[j]
            )
# ==============================================================
# 8) Translator O2_out
# ==============================================================
m.fs.translator_o2_out = Translator(
    inlet_property_package=m.fs.reaction_props,
    outlet_property_package=m.fs.air_props,
    outlet_state_defined=False,
)

m.fs.separator_o2rich_to_translator = Arc(
    source=m.fs.reactor_separator.o2_strm,
    destination=m.fs.translator_o2_out.inlet
)

comps_out = set(m.fs.translator_o2_out.properties_out[0].mole_frac_comp.keys())

@m.fs.translator_o2_out.Constraint(m.fs.time)
def flow_eqn_h2_out(b, t):
    return b.properties_out[t].flow_mol == b.properties_in[t].flow_mol

@m.fs.translator_o2_out.Constraint(m.fs.time)
def temp_eqn_h2_out(b, t):
    return b.properties_out[t].temperature == b.properties_in[t].temperature

@m.fs.translator_o2_out.Constraint(m.fs.time)
def press_eqn_h2_out(b, t):
    return b.properties_out[t].pressure == b.properties_in[t].pressure

@m.fs.translator_o2_out.Constraint(m.fs.time, comps_out)
def mole_frac_eqn_o2_out(b, t, j):
    if j == "O2":
        return b.properties_out[t].mole_frac_comp[j] == b.properties_in[t].mole_frac_comp[j]
    else: 
        return b.properties_out[t].mole_frac_comp[j] == 1e-19


# ==============================================================
# 10) Final mixer
# ==============================================================
m.fs.mixer_out = um.Mixer(
    inlet_list=["o2_strm_final", "o2_poor_strm"],
    momentum_mixing_type=um.MomentumMixingType.none,
    property_package=m.fs.air_props,
)
m.fs.mix_o2_final = Arc(source = m.fs.translator_o2_out.outlet, destination = m.fs.mixer_out.o2_strm_final)


@m.fs.mixer_out.Constraint(m.fs.time)
def pressure_eqn_mix_out(b, t):
    return b.mixed_state[t].pressure == b.o2_poor_strm_state[t].pressure

# ==============================================================
# 9) Heater for N2
# ==============================================================
m.fs.heater = um.Heater(
    property_package=m.fs.air_props,
    has_pressure_change=False,
)

m.fs.heater_in = Arc(source = m.fs.separator.o2_poor_strm, destination = m.fs.heater.inlet)
m.fs.mix_o2_poor = Arc(source = m.fs.heater.outlet, destination = m.fs.mixer_out.o2_poor_strm)
# Energy balance constraint
@m.fs.Constraint(m.fs.time)
def SOFC_energy_balance(b, t):
    return (
        b.heater.control_volume.heat[t]
        ==
        -b.reactor.control_volume.heat[t] - 9440
    )

pyo.TransformationFactory("network.expand_arcs").apply_to(m)


In [4]:
# ==============================================================
# 1) Translator H2
# ==============================================================
m.fs.translator_h2.initialize()
propagate_state(m.fs.mix_h2)
# ==============================================================
# 2) Air Separator
# ==============================================================
m.fs.separator.initialize()
propagate_state(m.fs.o2_to_translator)
# ==============================================================
# 3) Translator O2
# ==============================================================
m.fs.translator_o2.initialize()
propagate_state(m.fs.mix_o2)
# ==============================================================
# 4) Mixer (feed)
# ==============================================================
m.fs.mixer.initialize()
propagate_state(m.fs.mix_to_reactor)
# ==============================================================
# 5) Stoichiometric Reactor
# ==============================================================
m.fs.reactor.initialize()
propagate_state(m.fs.react_to_sep)
# ==============================================================
# 6) Product Separator
# ==============================================================
m.fs.reactor_separator.initialize()
propagate_state(m.fs.water_to_translator)
# ==============================================================
# 7) Translator H2_out
# ==============================================================
m.fs.translator_h2_out.initialize()
# ==============================================================
# 8) Translator O2_out
# ==============================================================
m.fs.translator_o2_out.initialize()
propagate_state(m.fs.separator_o2rich_to_translator)
# ==============================================================
# 9) Final mixer
# ==============================================================
m.fs.mixer_out.initialize()
propagate_state(m.fs.mix_o2_final)
propagate_state(m.fs.mix_o2_poor)
# ==============================================================
# 10) Heater
# ==============================================================
m.fs.heater.initialize()
propagate_state(m.fs.heater_in)


# --------------------------------------------------------------------------
# RISOLUZIONE DEL MODELLO COMPLETO
# --------------------------------------------------------------------------

iscale.calculate_scaling_factors(m)        # autoscale model

print("\n----- MODEL STATISTICS -----")
print(report_statistics(m))

solver = pyo.SolverFactory("ipopt")
solver.options = {"tol": 1e-6, "max_iter": 1000, "print_level": 5}

results = solver.solve(m, tee=True)

2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.mixer.fuel_strm_state[0.0].enth_mol_phase[Vap], for _enthalpy_flow_term
2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.mixer.o2_rich_strm_state[0.0].enth_mol_phase[Vap], for _enthalpy_flow_term
2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.mixer.mixed_state[0.0].enth_mol_phase[Vap], for _enthalpy_flow_term
2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.reactor.control_volume.properties_in[0.0].enth_mol_phase[Vap], for _enthalpy_flow_term
2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.reactor.control_volume.properties_out[0.0].enth_mol_phase[Vap], for _enthalpy_flow_term
2025-06-05 11:29:48 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.reactor.control_volume.rate_reaction_extent[0.0,h2_cmb]
2025-06-05 11:29:48 [WARNING] idaes.core.uti

In [37]:
import idaes_ui
import idaes_ui.fv
from idaes_ui.fv import visualize 
visualize(m.fs, "My Process Flowsheet")

2025-06-05 11:24:25 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 65197
2025-06-05 11:24:25 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'My Process Flowsheet.json'
2025-06-05 11:24:25 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'My Process Flowsheet.json' in current directory (c:\Users\Sara\Downloads)
2025-06-05 11:24:25 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='My Process Flowsheet' new='My-Process-Flowsheet'
Flowsheet name changed to 'My-Process-Flowsheet'
2025-06-05 11:24:25 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:65197/app?id=My-Process-Flowsheet


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000014E881492A0>, port=65197, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x0000014E875049D0>)

In [31]:
import pandas as pd

# ------------------------------------------------------------------
# helper: build a row for a given stream
# ------------------------------------------------------------------
def _row(label, s):
    comps = ["H2", "H2O", "O2", "N2"]          # add more if you wish
    r = dict(Stream=label,
             Flow_mol_s=value(s.flow_mol[0]),
             T_K=value(s.temperature[0]),
             P_bar=value(s.pressure[0])/1e5)
    for c in comps:
        r[f"x_{c}"] = value(s.mole_frac_comp[0, c]) if (0, c) in s.mole_frac_comp else None
    return r

# ------------------------------------------------------------------
# assemble the table
# ------------------------------------------------------------------
tbl = [
    _row("H2 feed",                    m.fs.translator_h2.outlet),
    _row("Air feed (O2-rich)",         m.fs.separator.o2_rich_strm),
    _row("Air purge (O2-poor)",        m.fs.separator.o2_poor_strm),
    _row("Mixer → Reactor",            m.fs.mixer.outlet),
    _row("Reactor inlet",              m.fs.reactor.inlet),
    _row("Reactor outlet",             m.fs.reactor.outlet),
    _row("Sep – O2 stream",            m.fs.reactor_separator.o2_strm),
    _row("Sep – H2O stream",           m.fs.reactor_separator.water_strm),
    _row("Recycle-H2 translator out",  m.fs.translator_h2_out.outlet),
    _row("O2 translator out",          m.fs.translator_o2_out.outlet),
    _row("Final stack gas",            m.fs.mixer_out.outlet),
]

df = pd.DataFrame(tbl)
pd.set_option("display.float_format", "{:,.6g}".format)   # nicer prints
display(df)                                               # Jupyter pretty-print
# df.to_excel("sofc_snapshot.xlsx", index=False)          # uncomment to save


,Stream,Flow_mol_s,T_K,P_bar,x_H2,x_H2O,x_O2,x_N2
0,H2 feed,0.075397,942.05,1.01325,0.99,0.01,2.036e-17,NaN
1,Air feed (O2-rich),0.42,948.45,1.01325,NaN,NaN,1,1e-20
2,Air purge (O2-poor),1.58,948.45,1.01325,NaN,NaN,1e-20,1
3,Mixer → Reactor,0.495397,947.585,1.01325,0.150673,0.00152195,0.847805,NaN
4,Reactor inlet,0.495397,947.585,1.01325,0.150673,0.00152195,0.847805,NaN
5,Reactor outlet,0.469272,"1,093.68",1.01325,0.0477184,0.11295,0.839332,NaN
6,Sep – O2 stream,0.393875,"1,093.68",1.01325,1e-20,1e-20,1,NaN
7,Sep – H2O stream,0.075397,"1,093.68",1.01325,0.297,0.703,1e-20,NaN
8,Recycle-H2 translator out,0.075397,"1,093.68",1.01325,0.297,0.703,NaN,NaN
9,O2 translator out,0.393875,"1,093.68",1.01325,NaN,NaN,1,1e-19


In [32]:
# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.translator_h2, "Translator H2 - Inlet", m.fs.translator_h2.inlet)
print_stream(m.fs.translator_h2, "Translator H2 - Outlet", m.fs.translator_h2.outlet)



# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator Air in o2_rich e o2_poor
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.separator, "Separator Air - Inlet", m.fs.separator.inlet)
print_stream(m.fs.separator, "Separator Air - O2 Rich", m.fs.separator.o2_rich_strm)
print_stream(m.fs.separator, "Separator Air - O2 Poor", m.fs.separator.o2_poor_strm)



# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: translator o2_rich da separator a reaction_props
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.translator_o2, "Translator O2 - Inlet", m.fs.translator_o2.inlet)
print_stream(m.fs.translator_o2, "Translator O2 - Outlet", m.fs.translator_o2.outlet)



# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per H2 e O2 inlet
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.mixer, "Mixer - O2 Inlet", m.fs.mixer.o2_rich_strm)
print_stream(m.fs.mixer, "Mixer - H2 Inlet", m.fs.mixer.fuel_strm)
print_stream(m.fs.mixer, "Mixer - Mixed Outlet", m.fs.mixer.outlet)



# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Stochiometric Reactor
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.reactor, "Reactor - Inlet", m.fs.reactor.inlet)
print_stream(m.fs.reactor, "Reactor - Outlet", m.fs.reactor.outlet)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator water and O2
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.reactor_separator, "Separator - O2 Stream", m.fs.reactor_separator.o2_strm)
print_stream(m.fs.reactor_separator, "Separator - Water Stream", m.fs.reactor_separator.water_strm)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator for H2O e H2 stream
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.translator_h2_out, "Translator H2 - Inlet", m.fs.translator_h2_out.inlet)
print_stream(m.fs.translator_h2_out, "Translator H2 - Outlet", m.fs.translator_h2_out.outlet)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2 stream
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.translator_o2_out, "Translator O2 - Inlet", m.fs.translator_o2_out.inlet)
print_stream(m.fs.translator_o2_out, "Translator O2 - Outlet", m.fs.translator_o2_out.outlet)


# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer for O2 and N2 stream from first separator
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.mixer_out, "Mixer at Outlet - O2 Inlet", m.fs.mixer_out.o2_strm_final)
print_stream(m.fs.mixer_out, "Mixer at Outlet - N2 Inlet", m.fs.mixer_out.o2_poor_strm)
print_stream(m.fs.mixer_out, "Mixer at Outlet- Mixed Outlet", m.fs.mixer_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Heater
# ----------------------------------------------------------------------------------------------------------------
print_stream(m.fs.mixer_out, "Heater - Inlet", m.fs.heater.inlet)
print_stream(m.fs.mixer_out, "Heater - Outlet", m.fs.heater.outlet)


                   Translator H2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_H2        : 9.900000e-01
  x_H2O       : 1.000000e-02

DOF after Translator H2 - Inlet: 0

                   Translator H2 - Outlet                   

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_O2        : 2.035996e-17
  x_H2        : 9.900000e-01
  x_H2O       : 1.000000e-02

DOF after Translator H2 - Outlet: 0

                   Separator Air - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 2.000000e+00
Temperature [K]     : 948.45
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_O2        : 2.100000e-01
  x_N2        : 7.900000e-01

DOF after Separator Air - Inlet: 0

                  Separator Air - O2 Rich                   

STREAM PROPERTIES:
Flow [mol/s]        :